# Basic functionality tests.

If the notebook cells complete with no exception the tests have passed.

The tests must be run in the full `jupyter notebook` or `jupyter lab` environment.

*Note:* I couldn't figure out to make the validation tests run correctly 
at top level cell evaluation using `Run all`
because the widgets initialize after later cells have executed, causing spurious
failures.  Consequently the automated validation steps involve an extra round trip using
a widget at the bottom of the notebook which is guaranteed to render last.

In [1]:
# Some test artifacts used below:

import jp_proxy_widget
from jp_proxy_widget import notebook_test_helpers

validators = notebook_test_helpers.ValidationSuite()

import time

class PythonClass:
    
    class_attribute = "initial class attribute value"
    
    def __init__(self):
        self.set_instance_attribute("initial instance attribute value")
        
    def set_instance_attribute(self, value):
        self.instance_attribute = value
        
    @classmethod
    def set_class_attribute(cls, value):
        cls.class_attribute = value

notebook_test_helpers

jp_proxy_widget


<module 'jp_proxy_widget' from '/Users/awatters/repos/jp_proxy_widget/jp_proxy_widget/__init__.py'>

In [2]:
python_instance = PythonClass()

def python_function(value1, value2):
    python_instance.new_attribute = "value1=%s and value2=%s" % (value1, value2)

# pong: test that a proxy widget can call back to Python

In [3]:
import jp_proxy_widget

In [4]:
pong = jp_proxy_widget.JSProxyWidget()

In [5]:
def validate_pong():
    # check that the Python callbacks were called.
    assert python_instance.instance_attribute == "instance"
    assert PythonClass.class_attribute == "class"
    assert python_instance.new_attribute == 'value1=1 and value2=3'
    assert pong.error_msg == 'No error'
    print ("pong says", pong.error_msg)
    print ("Pong callback test succeeded!")

pong.js_init("""
//debugger;
instance_method("instance");
class_method("class");
python_function(1, 3);
element.html("<b>Callback test widget: nothing interesting to see here</b>")
//validate()
""", 
             instance_method=python_instance.set_instance_attribute,
             class_method=PythonClass.set_class_attribute,
             python_function=python_function,
             #validate=validate_pong
            )

#widget_validator_list.append([pong, validate_pong])
validators.add_validation(pong, validate_pong)

#pong.debugging_display()
pong

JSProxyWidget(status='Not yet rendered')

In [ ]:

# set the mainloop check to True if running cells one at a time
mainloop_check = False

if mainloop_check:
    # At this time this fails on "run all"
    validate_pong()

# pingpong: test that Python can call in to a widget

... use a widget callback to pass the value back

In [ ]:
pingpong_list = "just some strings".split()

def pingpong_python_fn(argument1, argument2):
    print("called pingpong_python_fn") # this print goes nowhere?
    pingpong_list[:] = [argument1, argument2]

def validate_pingpong():
    # check that the callback got the right values
    assert pingpong_list == ["testing", 123]
    print ("ping pong test callback got ", pingpong_list)
    print ("ping pong test succeeded!")
    
pingpong = jp_proxy_widget.JSProxyWidget()
pingpong.js_init("""
element.html("<em>Ping pong test -- no call yet.</em>")
element.call_in_to_the_widget = function (argument1, argument2) {
    element.html("<b> Call in sent " + argument1 + " and " + argument2 + "</b>")
    call_back_to_python(argument1, argument2);
}
element.validate = validate;
""", call_back_to_python=pingpong_python_fn, validate=validate_pingpong)

#widget_validator_list.append([pingpong, validate_pingpong])
validators.add_validation(pingpong, validate_pingpong)

#pingpong.debugging_display()
pingpong

In [ ]:
# call in to javascript
pingpong.element.call_in_to_the_widget("testing", 123)
# call in to javascript and back to python to validate
pingpong.element.validate()

if mainloop_check:
    validate_pingpong()
    

# roundtrip: datatype round trip

Test that values can be passed in to the proxy widget and back out again.

In [ ]:
binary = bytearray(b"\x12\xff binary bytes")
string_value = "just a string"
int_value = -123
float_value = 45.6
json_dictionary = {"keys": None, "must": 321, "be": [6, 12], "strings": "values", "can": ["be", "any json"]}
list_value = [9, string_value, json_dictionary]

roundtrip_got_values = []

from jp_proxy_widget import hex_codec

def get_values_back(binary, string_value, int_value, float_value, json_dictionary, list_value):
    # NOTE: binary values must be converted explicitly from hex string encoding!
    binary = hex_codec.hex_to_bytearray(binary)
    roundtrip_got_values[:] = [binary, string_value, int_value, float_value, json_dictionary, list_value]

roundtrip_names = "binary string_value int_value float_value json_dictionary list_value".split()

def validate_roundtrip():
    #assert roundtrip_got_values == [string_value, int_value, float_value, json_dictionary, list_value]
    expected_values = [binary, string_value, int_value, float_value, json_dictionary, list_value]
    assert len(expected_values) == len(roundtrip_got_values)
    for (name, got, expected) in zip(roundtrip_names, roundtrip_got_values, expected_values):
        assert got == expected, "values don't match: " + repr((name, got, expected))
    print ("roundtrip values match!")
    
roundtrip = jp_proxy_widget.JSProxyWidget()
roundtrip.js_init(r"""
element.all_values = [binary, string_value, int_value, float_value, json_dictionary, list_value];
html = ["<pre> Binary values sent as bytearrays appear in Javascript as Uint8Arrays"]
for (var i=0; i<names.length; i++) {
    html.push(names[i]);
    var v = element.all_values[i];
    if (v instanceof Uint8Array) {
        html.push("    Uint8Array")
    } else {
        html.push("    type: " + (typeof v))
    }
    html.push("    value: " + v);
}
html.push("</pre>");
element.html(html.join("\n"));

// send the values back
callback(binary, string_value, int_value, float_value, json_dictionary, list_value);
""",
                 binary=binary,
                 string_value=string_value,
                 int_value=int_value,
                 float_value=float_value,
                 json_dictionary=json_dictionary,
                 list_value=list_value,
                 names=roundtrip_names,
                 callback=get_values_back,
                 # NOTE: must up the callable level!
                 callable_level=4
                )

roundtrip

In [ ]:
validators.add_validation(roundtrip, validate_roundtrip)

if mainloop_check:
    validate_roundtrip()

# loadCSS -- test load of simple CSS file.

We want to load this css file

In [ ]:
from jp_proxy_widget import js_context
style_fn="js/simple.css"
print(js_context.get_text_from_file_name(style_fn))

In [ ]:
loadCSS = jp_proxy_widget.JSProxyWidget()

# load the file
loadCSS.load_css(style_fn)

# callback for storing the styled element color
loadCSSstyle = {}

def color_callback(color):
    loadCSSstyle["color"] = color

# initialize the element using the style and callback to report the color.
loadCSS.js_init("""
element.html('<div><em class="random-style-for-testing" id="loadCSSelement">Styled widget element.</em></div>')

var e = document.getElementById("loadCSSelement");
var style = window.getComputedStyle(e);
color_callback(style["color"]);
""", color_callback=color_callback)

def validate_loadCSS():
    expect = 'rgb(216, 50, 61)'
    assert expect == loadCSSstyle["color"], repr((expect, loadCSSstyle))
    print ("Loaded CSS color is correct!")

loadCSS

In [ ]:
validators.add_validation(loadCSS, validate_loadCSS)

if mainloop_check:
    validate_loadCSS()
    

# loadJS -- load a javascript file (once only per interpreter)

We want to load this javascript file:

In [ ]:
js_fn="js/simple.js"
print(js_context.get_text_from_file_name(js_fn))

In [ ]:
loadJS = jp_proxy_widget.JSProxyWidget()

# load the file
loadJS.load_js_files([js_fn], force=True)

# callback for storing the styled element color
loadJSinfo = {}

def answer_callback(answer):
    loadJSinfo["answer"] = answer

loadJS.js_init("""
    element.html('<b>The answer is ' + window.the_answer + '</b>')
    answer_callback(window.the_answer);
""", answer_callback=answer_callback, js_fn=js_fn)

def validate_loadJS():
    expect = 42
    assert expect == loadJSinfo["answer"], repr((expect, loadJSinfo))
    print ("Loaded JS value is correct!")

loadJS

In [ ]:
validators.add_validation(loadJS, validate_loadJS)

if mainloop_check:
    validate_loadJS()
    
loadJS.print_status()

In [ ]:
delay_ms = 1000
validators.run_all_in_widget(delay_ms=delay_ms)